In [5]:
import boto3
import numpy 
import sagemaker
from sagemaker.pytorch import PyTorch
import torch
import os

### DENSENET 211

In [6]:
# Sagemaker Notebook must be of type, conda_pytorch_p36

!pip install -r '/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/requirements.txt'

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
# Need to add this to requirements.txt
!pip install tensorboard

fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Train the model per the settings specified in the original paper
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    train.batch_size 64 \
    train.base_lr 0.1 \
    dataset.name CIFAR10_RA_2_5 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00 \
    scheduler.epochs 400

[2020-07-01 02:03:57] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10_RA_2_5
  dataset_dir: ''
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_type: bottleneck

In [8]:
# Resume training with the un-augmented data
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python train.py --config /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/config.yaml \
    train.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00400.pth \
    dataset.name CIFAR10 \
    train.base_lr 0.001 \
    train.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50 \
    scheduler.epochs 50


[2020-07-02 04:25:15] __main__ INFO: device: cuda
cudnn:
  benchmark: True
  deterministic: False
dataset:
  name: CIFAR10
  dataset_dir: ~/.torch/datasets/CIFAR10
  image_size: 32
  n_channels: 3
  n_classes: 10
model:
  type: cifar
  name: densenet
  init_mode: kaiming_fan_out
  vgg:
    n_channels: [64, 128, 256, 512, 512]
    n_layers: [2, 2, 3, 3, 3]
    use_bn: True
  resnet:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
  resnet_preact:
    depth: 110
    n_blocks: [2, 2, 2, 2]
    block_type: basic
    initial_channels: 16
    remove_first_relu: False
    add_last_bn: False
    preact_stage: [True, True, True]
  wrn:
    depth: 28
    initial_channels: 16
    widening_factor: 10
    drop_rate: 0.0
  densenet:
    depth: 100
    n_blocks: [6, 12, 24, 16]
    block_type: bottleneck
    growth_rate: 12
    drop_rate: 0.0
    compression_rate: 0.5
  pyramidnet:
    depth: 272
    n_blocks: [3, 24, 36, 3]
    initial_channels: 16
    block_

In [9]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/test_results_0050_cifar10

[2020-07-02 12:37:00] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.04it/s]
[2020-07-02 12:37:20] __main__ INFO: Elapsed 19.52
[2020-07-02 12:37:20] __main__ INFO: Loss 0.2990 Accuracy 0.9321


In [10]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00400.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/test_results_0400_cifar10

[2020-07-02 12:38:05] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00400.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.02it/s]
[2020-07-02 12:38:26] __main__ INFO: Elapsed 19.57
[2020-07-02 12:38:26] __main__ INFO: Loss 0.5422 Accuracy 0.8907


In [11]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/test_results_0050_cifar101

[2020-07-02 12:40:02] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00_resume400_50/checkpoint_00050.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.52it/s]
[2020-07-02 12:40:07] __main__ INFO: Elapsed 4.26
[2020-07-02 12:40:07] __main__ INFO: Loss 0.5555 Accuracy 0.8570


In [12]:
## Evaluate the trained, saved model using the CIFAR 10.1 test dataset 
# Write the results to the test output directory specified.
os.chdir('/home/ec2-user/SageMaker/w210-capstone/models/pytorch_imageclass/')
!python evaluate.py --config configs/cifar/densenet.yaml \
    model.densenet.depth 100 \
    model.densenet.growth_rate 12 \
    test.batch_size 64 \
    dataset.name CIFAR101 \
    test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00400.pth \
    test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/test_results_0400_cifar101

[2020-07-02 12:40:55] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00400.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.46it/s]
[2020-07-02 12:41:00] __main__ INFO: Elapsed 4.29
[2020-07-02 12:41:00] __main__ INFO: Loss 1.1260 Accuracy 0.7770


In [13]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/test_results_0300_cifar10

[2020-07-02 12:41:33] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00300.pth
Files already downloaded and verified
100%|█████████████████████████████████████████| 157/157 [00:19<00:00,  8.05it/s]
[2020-07-02 12:41:54] __main__ INFO: Elapsed 19.50
[2020-07-02 12:41:54] __main__ INFO: Loss 0.5343 Accuracy 0.8911


In [14]:
## Evaluate the trained, saved model using the CIFAR 10 test dataset 
# Write the results to the test output directory specified.
!python evaluate.py --config configs/cifar/densenet.yaml \
   model.densenet.depth 100 \
   model.densenet.growth_rate 12 \
   test.batch_size 64 \
   dataset.name CIFAR101 \
   test.checkpoint /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00300.pth \
   test.output_dir /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/test_results_0300_cifar101

[2020-07-02 12:42:29] fvcore.common.checkpoint INFO: Loading checkpoint from /home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/checkpoint_00300.pth
CIFAR 10.1
100%|███████████████████████████████████████████| 32/32 [00:04<00:00,  7.46it/s]
[2020-07-02 12:42:33] __main__ INFO: Elapsed 4.29
[2020-07-02 12:42:33] __main__ INFO: Loss 1.1397 Accuracy 0.7840


In [ ]:
# Write the results to a CSV file so that we can analyze later.
import pandas as pd

results = {'Model': ['densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12', 'densenet_BC_100_12'],
           'Testset': ['cifar10', 'cifar10', 'cifar10', 'cifar10.1'],
           'Epoch': [100, 200, 300, 300],
           'Loss': [0.3681, 0.2279, 0.2223, 0.5098],
           'Accuracy': [0.8875, 0.9456, 0.9484, 0.8830],
           'Original_Accuracy': [95.5, 95.5, 95.5, 87.6],
           'Original_CI': [(95.1, 95.9), (95.1, 95.9), (95.1, 95.9), (86.1, 89.0)]
           }

df = pd.DataFrame(results, columns = ['Model', 'Testset', 'Epoch', 'Loss', 'Accuracy', 
                                      'Original_Accuracy', 'Original_CI'])


df.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12/exp00/results.csv')
df.head()

In [16]:
import pandas as pd
a = pd.Series(['densenet_BC_100_12_ra_2_5_c10val', 400, 'cifar10', 0.5422, 0.8907])
b = pd.Series(['densenet_BC_100_12_ra_2_5_c10val', 300, 'cifar10', 0.5343, 0.8911])
c = pd.Series(['densenet_BC_100_12_ra_2_5_c10val', 400, 'cifar10.1', 1.1260, 0.7770])
d = pd.Series(['densenet_BC_100_12_ra_2_5_c10val', 300, 'cifar10.1', 1.1397, 0.7840])

e = pd.Series(['densenet_BC_100_12_ra_2_5_refined400_c10val', 50, 'cifar10.1', 0.5555, 0.8570])
f = pd.Series(['densenet_BC_100_12_ra_2_5_refined400_c10val', 50, 'cifar10', 0.2990, 0.9321])
# g = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10', 0.4499, 0.8795])
# h = pd.Series(['resnet_basic_32_ra_2_5_refined300', 150, 'cifar10.1', 0.8206, 0.7710])
               
df_results = pd.concat([a,b,c,d,e,f], axis=1).T
df_results.columns = ['Model', 'Epoch', 'Testset', 'Loss', 'Accuracy']

df_results['Original_Accuracy'] = df_results.apply((lambda row: 95.5 if row[2] == 'cifar10' else 87.6), axis=1)
df_results['Original_CI'] = df_results.apply((lambda row: (95.1, 95.9) if row[2] == 'cifar10' else (86.1, 89.0)), axis=1)

df_results.to_csv('/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/results.csv')
df_results

,Model,Epoch,Testset,Loss,Accuracy,Original_Accuracy,Original_CI
0,densenet_BC_100_12_ra_2_5_c10val,400,cifar10,0.5422,0.8907,95.5,"(95.1, 95.9)"
1,densenet_BC_100_12_ra_2_5_c10val,300,cifar10,0.5343,0.8911,95.5,"(95.1, 95.9)"
2,densenet_BC_100_12_ra_2_5_c10val,400,cifar10.1,1.126,0.777,87.6,"(86.1, 89.0)"
3,densenet_BC_100_12_ra_2_5_c10val,300,cifar10.1,1.1397,0.784,87.6,"(86.1, 89.0)"
4,densenet_BC_100_12_ra_2_5_refined400_c10val,50,cifar10.1,0.5555,0.857,87.6,"(86.1, 89.0)"
5,densenet_BC_100_12_ra_2_5_refined400_c10val,50,cifar10,0.299,0.9321,95.5,"(95.1, 95.9)"


In [17]:
# Peak inside the output file for predictions
import numpy as np
output = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val/exp00/test_results_0400_cifar10/predictions.npz'
npzfile = np.load(output)
print(npzfile.files)
npzfile['preds']

['preds', 'probs', 'labels', 'loss', 'acc']


array([[-5.0915952e+00, -2.9502690e+00,  5.6365705e+00, ...,
        -2.1884670e+00,  3.3478710e-01, -9.9007330e+00],
       [-1.6314175e+00,  5.5563874e+00, -3.7057674e+00, ...,
        -3.8177397e+00,  2.2469038e+01, -3.4152243e+00],
       [ 2.2127066e+00,  5.2995901e+00,  3.3887062e+00, ...,
        -1.8843362e+00,  9.3164425e+00,  8.7492657e-01],
       ...,
       [ 1.6178796e+00, -9.5727987e+00,  4.7883625e+00, ...,
        -8.4454066e-01, -3.5555537e+00, -7.6539884e+00],
       [ 1.4942882e-02,  6.2370715e+00, -5.8590088e-02, ...,
        -1.0772719e+00, -2.5271067e-01,  3.0723653e+00],
       [-1.5226712e+00,  2.9217884e-01, -2.2915471e+00, ...,
         1.2915038e+01,  9.8799825e-01, -7.6552629e-01]], dtype=float32)

In [17]:
# Upload the model checkpoints, configs, and results to S3 
bucket='sagemaker-june29'
prefix = 'sagemaker/results/original-models/densenet_BC_100_12_ra_2_5_c10val'
path = '/home/ec2-user/SageMaker/experiments/densenet_BC_100_12_ra_2_5_c10val'

s3_resource = boto3.resource("s3", region_name="us-east-2")

def uploadDirectory(local_path,bucket_name,s3_prefix):

    my_bucket = s3_resource.Bucket(bucket_name)
    
    for path, subdirs, files in os.walk(local_path):
        path = path.replace("\\","/")
        directory_name = path.replace(local_path,"")
        for file in files:
            #print("Local File:", os.path.join(path, file))
            #print("      Dest:", s3_prefix+directory_name+'/'+file)
            my_bucket.upload_file(os.path.join(path, file), s3_prefix+directory_name+'/'+file)
    
uploadDirectory(path,bucket,prefix)